## Vector Space

 1. take unigram term-document vectors from n-gram matrix
 2. apply PCA (or UMAP directly?) for dimensionality reduction (original vectors have dimensionality > 10k)
 3. write vectors to .tsv -- also write vector features to file (term, avg pmi?, ...) -> for visualisation in embedding projector
 4. use KDTree (or KMeans?; -> elbow curve) for clusters

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd


from sklearn.feature_extraction.text import TfidfTransformer
from utils import Ngram

NG = Ngram.load("counts", precompute_freqs=True)


tfidf = TfidfTransformer()


ti_mat = tfidf.fit_transform(NG.term_doc_matrix)

None-grams: Term frequencies precomputed...
None-grams: Init done


In [13]:
# uni_inds = NG.vocab(1, with_inds=True)

# sub_vocab = {w: i for w, i in zip(sorted(uni_inds, key=lambda w: uni_inds[w]),
#                                  range(len(uni_inds)))}

uni_voc = NG.vocab(1, with_inds=True)

r = len(uni_inds)//2

sub_voc = sorted(np.random.choice(list(uni_voc.keys()), r))

sub_inds = [uni_voc[w] for w in sub_voc]

sub_mat = ti_mat[:, sub_inds].T.toarray()

In [14]:
len(NG.vocab(1)), sub_mat.shape

(27104, (13552, 11199))

In [18]:
from sklearn.decomposition import PCA

pca = PCA(n_components=512)
Y = pca.fit_transform(sub_mat)

In [ ]:
from umap import UMAP


umap = UMAP(n_neighbors=15, n_components=512, verbose=100)

In [ ]:
Y = umap.fit_transform(sub_mat)
Y.shape

### Write to TSV

In [21]:
v_to_tsv = lambda v: "\t".join(map(str, v))

# fewer_vecs = np.random.permutation(list(vecs.items()))[:5000]

with open(f"vecs_pca_{len(sub_inds)}.tsv", "w") as handle:
    for v in tqdm(Y):
        handle.write(v_to_tsv(v))
        handle.write("\n")

100%|██████████| 13552/13552 [00:05<00:00, 2326.57it/s]


In [20]:
with open("meta.tsv", "w") as handle:
    for w in sub_voc:
        handle.write(str(w)+"\n")

## Clusters

In [ ]:
from sklearn.neighbors import KDTree

tree = KDTree(vectors.toarray(), metric="l2")